In [1]:
import pysftp
import os 
import numpy as np
import pandas as pd
import netCDF4
import numpy as np
import matplotlib.pyplot as plt


from scipy import io
from classes.matlab_reader import matlab_reader
from classes.grid_funcs import map_grid,dataset,data_wizard
from dotenv import load_dotenv   #for python-dotenv method
from datetime import datetime

pd.set_option('display.max_rows', 60)


/Users/joaojesus/opt/anaconda3/envs/geo_plotting/lib/python3.8/site-packages/paramiko/transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [24]:
YEAR_ANALYSIS = '2018'

In [25]:
# Get Wind Data

path = '/Users/joaojesus/Desktop/FinalYearProject/GENERATED_CSV/WIND_EASTWARD/level_100/'
grid = map_grid(grid_size=None, source = 'era5')
wizard = data_wizard()
res_mean = []
res_mean_selection = []
file_names = []
for file in os.listdir(path):
    if file[5:9] == YEAR_ANALYSIS: 
        wind_eastward = np.loadtxt(path + file, delimiter=",")

        select_area_indexer = grid.select_area_indexer(
            min_lat= 55,
            max_lat= 65,
            min_lon= -180,
            max_lon= 180)

        data = wizard.data_filtering(select_area_indexer = select_area_indexer ,data = wind_eastward)

        file_names.append(file)
        res_mean.append(np.nanmean(wind_eastward))
        res_mean_selection.append(np.nanmean(data))

df = pd.DataFrame(data={
    'file' : file_names,
    'wind_eastward_mean' : res_mean,
    'wind_eastward_mean_selection' : res_mean_selection
})


year = [string[5:9] for string in df['file'].to_list()]
day = [string[10:13] for string in df['file'].to_list()]

import datetime
dates = []

for i in range(len(day)):
    date = datetime.datetime(int(year[i]), 1, 1)
    date += datetime.timedelta(days=int(day[i])-1)
    dates.append(date)

df['dates'] = dates
df['dates'] = pd.to_datetime(df['dates'])

df = df.sort_values(by='dates')

df = df.reset_index()
df = df.drop(columns=['index'])

# Define the outlier threshold
outlier_threshold = df['wind_eastward_mean_selection'].mean() + (df['wind_eastward_mean_selection'].std() * 3)

df['outliers'] = df['wind_eastward_mean_selection'] > outlier_threshold

In [26]:
# Define a function to apply to each row of the DataFrame
def replace_outliers(row):
    if not row['outliers']:
        # If B is False, return the value of A
        return row['wind_eastward_mean_selection']
    else:
        # If B is True, replace the value of A with the average of the last 3 values of A
        last_3_avg = df.loc[:row.name, 'wind_eastward_mean_selection'].tail(5).mean()
        return last_3_avg

# Apply the function to each row of the DataFrame and store the result in a new column 'C'
df['res_filtered'] = df.apply(replace_outliers, axis=1)

values = df['res_filtered'].to_list()
SSW_ = [False]

for i in range(1,len(values)):
    if values[i-1] >= 0 and values[i] < 0:
        SSW_.append(True)
    else:
        SSW_.append(False)

df['SSW_'] = SSW_

df['SSW_flag'] = False
day_threshold = 7
for index in df[df['SSW_'] == True].index:
    if len(df.iloc[index:index+day_threshold].loc[df['res_filtered'] < -0.1]) >= day_threshold :
        df['SSW_flag'].iloc[index] = True

In [27]:
# Get Temperature Data

path = '/Users/joaojesus/Desktop/FinalYearProject/GENERATED_CSV/TEMPERATURE/level_100/'
grid = map_grid(grid_size=None, source = 'era5')
wizard = data_wizard()
res_mean = []
res_mean_selection = []
file_names = []
for file in os.listdir(path):
    if file[5:9] == YEAR_ANALYSIS: 
        temperature = np.loadtxt(path + file, delimiter=",")

        select_area_indexer = grid.select_area_indexer(
            min_lat= 60,
            max_lat= 90,
            min_lon= -180,
            max_lon= 180)

        data = wizard.data_filtering(select_area_indexer = select_area_indexer ,data = temperature)

        file_names.append(file)
        res_mean.append(np.nanmean(temperature))
        res_mean_selection.append(np.nanmean(data))

df_temperature = pd.DataFrame(data={
    'file' : file_names,
    'temperature' : res_mean,
    'temperature_selection_mean' : res_mean_selection
})


year = [string[5:9] for string in df_temperature['file'].to_list()]
day = [string[10:13] for string in df_temperature['file'].to_list()]

import datetime
dates = []

for i in range(len(day)):
    date = datetime.datetime(int(year[i]), 1, 1)
    date += datetime.timedelta(days=int(day[i])-1)
    dates.append(date)

df_temperature['dates'] = dates
df_temperature['dates'] = pd.to_datetime(df_temperature['dates'])

df_temperature = df_temperature.sort_values(by='dates')

df_temperature = df_temperature.reset_index()
df_temperature = df_temperature.drop(columns=['index'])


In [28]:
df_temperature

,file,temperature,temperature_selection_mean,dates
0,era5_2018d001.nc_3_100_temperature.csv,259.608432,242.769261,2018-01-01
1,era5_2018d002.nc_3_100_temperature.csv,259.501266,242.309842,2018-01-02
2,era5_2018d003.nc_3_100_temperature.csv,259.556069,241.995566,2018-01-03
3,era5_2018d004.nc_3_100_temperature.csv,259.636655,241.612526,2018-01-04
4,era5_2018d005.nc_3_100_temperature.csv,259.672683,241.308549,2018-01-05
5,era5_2018d006.nc_3_100_temperature.csv,259.686315,241.379174,2018-01-06
6,era5_2018d007.nc_3_100_temperature.csv,259.830714,241.913818,2018-01-07
7,era5_2018d008.nc_3_100_temperature.csv,259.928507,242.961023,2018-01-08
8,era5_2018d009.nc_3_100_temperature.csv,260.109777,244.331364,2018-01-09
9,era5_2018d010.nc_3_100_temperature.csv,260.249701,244.991246,2018-01-10


In [29]:
df = pd.merge(df,df_temperature[['dates','temperature_selection_mean']],how='left', on='dates')

In [31]:
import plotly.graph_objects as go

# Create figure and plot the DataFrame
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['dates'], y=df['res_filtered'], name='Res Filtered', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=df['dates'], y=df['temperature_selection_mean'], name='Temperature Selection Mean', line=dict(color='orange'), yaxis='y2'))

# Add a dashed line at y=0
fig.add_shape(type='line', x0=df['dates'].min(), y0=0, x1=df['dates'].max(), y1=0, line=dict(color='gray', dash='dash'))

# Find the index of the first negative value
first_negative_index = (df['res_filtered'] < 0).idxmin()

# Add vertical lines for SSW_flag == True
for i, row in df.iterrows():
    if row.SSW_flag == True:
        fig.add_shape(type='line', x0=row.dates, y0=-50, x1=row.dates, y1=50, line=dict(color='red', dash='dash'))

# Set layout
fig.update_layout(
    yaxis=dict(
        range=[-50, 50],
        title='Res Filtered'
    ),
    yaxis2=dict(
        range=[230, 270],
        title='Temperature Selection Mean',
        side='right',
        overlaying='y'
    ),
    xaxis=dict(title='Dates'),
    title=f'ZMZW at 60◦ N latitude in {YEAR_ANALYSIS}',
    legend=dict(orientation="h", y=1.2, x=0.5)
)

# Show the plot
fig.show()

In [ ]:
row['dates']

Timestamp('2017-01-01 00:00:00')